#for colab

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [0]:
!pip uninstall pyglet
!pip install --user pyglet==1.3.2

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

W0726 09:16:21.509313 140352688899968 abstractdisplay.py:151] xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1033'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1033'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
# env = wrap_env(gym.make("MsPacman-v0"))

#main

In [0]:
import gym
import numpy as np
import time

In [0]:
from tqdm import tqdm

In [0]:
def digitize_state(observation):
    p, v, a, w = observation
    d = num_digitized
    pn = np.digitize(p, np.linspace(-2.4, 2.4, d+1)[1:-1])
    vn = np.digitize(v, np.linspace(-3.0, 3.0, d+1)[1:-1])
    an = np.digitize(a, np.linspace(-0.5, 0.5, d+1)[1:-1])
    wn = np.digitize(w, np.linspace(-2.0, 2.0, d+1)[1:-1])
    return pn + vn*d + an*d**2 + wn*d**3
  
def get_action(next_state, episode):
    epsilon = 0.5 * (1 / (episode + 1))
    if epsilon <= np.random.uniform(0, 1):
        a = np.where(q_table[next_state]==q_table[next_state].max())[0]
        next_action = np.random.choice(a)
    else:
        next_action = np.random.choice([0, 1])
    return next_action
  
def update_Qtable(q_table, state, action, reward, next_state):
    gamma = 0.99
    alpha = 0.5
    next_maxQ = max(q_table[next_state])
    q_table[state, action] = (1-alpha)*q_table[state, action] + alpha*(reward+gamma*next_maxQ)
    return q_table

In [0]:
env = wrap_env(gym.make('CartPole-v0'))
max_number_of_steps = 200 # 1試行のstep数
num_episodes = 1000 # 総試行回数
num_digitized = 6 # 分割数

In [0]:
q_table = np.random.uniform(low=-1, high=1, size=(num_digitized**4, env.action_space.n))

In [0]:
for tqdm(episode in range(num_episodes)):
    observation = env.reset()
    state = digitize_state(observation)
    action = np.argmax(q_table[state])
    episode_reward = 0
    
    for t in range(max_number_of_steps):
#         if episode % 10 == 0:
#             env.render()
        observation, reward, done, info = env.step(action)
        if done and t < max_number_of_steps-1:
            reward -= max_number_of_steps # 棒が倒れたら罰則
        episode_reward += reward
        next_state = digitize_state(observation) # t+1での観測状態を, 離散値に変換
        q_table = update_Qtable(q_table, state, action, reward, next_state)
        action = get_action(next_state, episode) # a_{t+1}
        state = next_state
        if done:
            break
    print('episode: ', episode, 'R: ', episode_reward)
    
np.savetxt('Qvalue.txt', q_table)

In [21]:
env.render()
show_video()